# 第4章 变形
- 声明：本文绝大部分内容摘自datawhale 学习教程

In [18]:
import numpy as np
import pandas as pd
df = pd.read_csv('joyful-pandas-master/data/table.csv')
df.head()

,School,Class,ID,Gender,Address,Height,Weight,Math,Physics
0,S_1,C_1,1101,M,street_1,173,63,34.0,A+
1,S_1,C_1,1102,F,street_2,192,73,32.5,B+
2,S_1,C_1,1103,M,street_2,186,82,87.2,B+
3,S_1,C_1,1104,F,street_2,167,81,80.4,B-
4,S_1,C_1,1105,F,street_4,159,64,84.8,B+


### 一、透视表
#### 1. pivot
- 一般状态下，数据在DataFrame会以压缩（stacked）状态存放，例如上面的Gender，两个类别被叠在一列中，pivot函数可将某一列作为新的cols：

In [2]:
df.pivot(index='ID',columns='Gender',values='Height').head()    #相当于EXCEL透视表行，列，数值计算

Gender,F,M
ID,,
1101,NaN,173.0
1102,192.0,NaN
1103,NaN,186.0
1104,167.0,NaN
1105,159.0,NaN


- 然而pivot函数具有很强的局限性，除了功能上较少之外，还不允许values中出现重复的行列索引对（pair），例如下面的语句就会报错：

In [5]:
#df.pivot(index='School',columns='Gender',values='Height').head()   #因为行列出现了重复，所以报错

- 因此，更多的时候会选择使用强大的pivot_table函数

#### 2. pivot_table
- 首先，再现上面的操作：

In [6]:
pd.pivot_table(df,index='ID',columns='Gender',values='Height').head()  #pivot 的功能，pivot_table也具备

Gender,F,M
ID,,
1101,NaN,173.0
1102,192.0,NaN
1103,NaN,186.0
1104,167.0,NaN
1105,159.0,NaN


In [17]:
df.pivot_table(index='School',columns='Gender',values='Height').head()   #pivot_table还可以突破pivot的局限性

Gender,F,M
School,,
S_1,173.125000,178.714286
S_2,173.727273,172.000000


- 由于功能更多，速度上自然是比不上原来的pivot函数：

In [10]:
%timeit df.pivot(index='ID',columns='Gender',values='Height')
%timeit pd.pivot_table(df,index='ID',columns='Gender',values='Height')

1.29 ms ± 2.53 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
5.36 ms ± 28 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


**Pandas中提供了各种选项，下面介绍常用参数：**
#### 1)aggfunc：对组内进行聚合统计，可传入各类函数，默认为'mean'

In [14]:
pd.pivot_table(df,index='School',columns='Gender',values='Height',aggfunc=['mean','sum']).head() #mean 函数是求均值

mean               sum      
Gender           F           M     F     M
School                                    
S_1     173.125000  178.714286  1385  1251
S_2     173.727273  172.000000  1911  1548

#### 2)margins：汇总边际状态

In [18]:
pd.pivot_table(df,index='School',columns='Gender',values='Height',aggfunc=['mean','sum'],margins=True).head()
#margins_name可以设置名字，默认为'All'

mean                           sum            
Gender           F           M         All     F     M   All
School                                                      
S_1     173.125000  178.714286  175.733333  1385  1251  2636
S_2     173.727273  172.000000  172.950000  1911  1548  3459
All     173.473684  174.937500  174.142857  3296  2799  6095

#### 3）行、列、值都可以为多级

In [19]:
pd.pivot_table(df,index=['School','Class'],
               columns=['Gender','Address'],
               values=['Height','Weight'])

Height                                                        \
Gender              F                                                     M   
Address      street_1 street_2 street_4 street_5 street_6 street_7 street_1   
School Class                                                                  
S_1    C_1        NaN    179.5    159.0      NaN      NaN      NaN    173.0   
       C_2        NaN      NaN    176.0    162.0    167.0      NaN      NaN   
       C_3      175.0      NaN      NaN    187.0      NaN      NaN      NaN   
S_2    C_1        NaN      NaN      NaN    159.0    161.0      NaN      NaN   
       C_2        NaN      NaN      NaN      NaN      NaN    188.5    175.0   
       C_3        NaN      NaN    157.0      NaN    164.0    190.0      NaN   
       C_4        NaN    176.0      NaN      NaN    175.5      NaN      NaN   

                                         ...   Weight                    \
Gender                                   ...        F                     
Address      street_2 street_4 street_5  ... street_4 street_5 street_6   
School Class                             ...                              
S_1    C_1      186.0      NaN      NaN  ...     64.0      NaN      NaN   
       C_2        NaN      NaN    188.0  ...     94.0     63.0     63.0   
       C_3      195.0    161.0      NaN  ...      NaN     69.0      NaN   
S_2    C_1        NaN    163.5      NaN  ...      NaN     97.0     61.0   
       C_2        NaN    155.0    193.0  ...      NaN      NaN      NaN   
       C_3        NaN    187.0    171.0  ...     78.0      NaN     81.0   
       C_4        NaN      NaN      NaN  ...      NaN      NaN     57.0   

                                                                             
Gender                       M                                               
Address      street_7 street_1 street_2 street_4 street_5 street_6 street_7  
School Class                                                                 
S_1    C_1        NaN     63.0     82.0      NaN      NaN      NaN      NaN  
       C_2        NaN      NaN      NaN      NaN     68.0     53.0      NaN  
       C_3        NaN      NaN     70.0     68.0      NaN      NaN     82.0  
S_2    C_1        NaN      NaN      NaN     71.0      NaN      NaN     84.0  
       C_2       76.5     74.0      NaN     91.0    100.0      NaN      NaN  
       C_3       99.0      NaN      NaN     73.0     88.0      NaN      NaN  
       C_4        NaN      NaN      NaN      NaN      NaN      NaN     82.0  

[7 rows x 24 columns]

#### 3.crosstab（交叉表）
- 交叉表是一种特殊的透视表，典型的用途如分组统计，如现在想要统计关于街道和性别分组的频数：

In [20]:
pd.crosstab(index=df['Address'],columns=df['Gender'])

Gender,F,M
Address,,
street_1,1,2
street_2,4,2
street_4,3,5
street_5,3,3
street_6,5,1
street_7,3,3


- 交叉表的功能也很强大（但目前还不支持多级分组），下面说明一些重要参数：

#### 1)values和aggfunc：分组对某些数据进行聚合操作，这两个参数必须成对出现

In [22]:
pd.crosstab(index=df['Address'],columns=df['Gender'],
            values=np.random.randint(1,20,df.shape[0]),aggfunc='min')
#默认参数等于如下方法：
#pd.crosstab(index=df['Address'],columns=df['Gender'],values=1,aggfunc='count')

Gender,F,M
Address,,
street_1,5,6
street_2,5,2
street_4,5,4
street_5,4,1
street_6,8,7
street_7,13,3


#### 2)除了边际参数margins外，还引入了normalize参数，可选'all','index','columns'参数值

In [24]:
pd.crosstab(index=df['Address'],columns=df['Gender'],normalize='all',margins=True)

Gender,F,M,All
Address,,,
street_1,0.028571,0.057143,0.085714
street_2,0.114286,0.057143,0.171429
street_4,0.085714,0.142857,0.228571
street_5,0.085714,0.085714,0.171429
street_6,0.142857,0.028571,0.171429
street_7,0.085714,0.085714,0.171429
All,0.542857,0.457143,1.000000


### 一、其他变形方法
#### 1. melt
- melt函数可以认为是pivot函数的逆操作，将unstacked状态的数据，压缩成stacked，使“宽”的DataFrame变“窄”

In [25]:
df_m = df[['ID','Gender','Math']]
df_m.head()

,ID,Gender,Math
0,1101,M,34.0
1,1102,F,32.5
2,1103,M,87.2
3,1104,F,80.4
4,1105,F,84.8


In [26]:
df.pivot(index='ID',columns='Gender',values='Math').head()

Gender,F,M
ID,,
1101,NaN,34.0
1102,32.5,NaN
1103,NaN,87.2
1104,80.4,NaN
1105,84.8,NaN


- melt函数中的id_vars表示需要保留的列，value_vars表示需要stack的一组列

In [28]:
pivoted = df.pivot(index='ID',columns='Gender',values='Math')
result = pivoted.reset_index().melt(id_vars=['ID'],value_vars=['F','M'],value_name='Math')\
                     .dropna().set_index('ID').sort_index()
#检验是否与展开前的df相同，可以分别将这些链式方法的中间步骤展开，看看是什么结果
result.equals(df_m.set_index('ID'))

True

#### 2.压缩与展开
#### 1) stack：这是最基础的变形函数，总共只有两个参数：level和dropna

In [29]:
df_s = pd.pivot_table(df,index=['Class','ID'],columns='Gender',values=['Height','Weight'])
df_s.groupby('Class').head(2)

Height        Weight      
Gender          F      M      F     M
Class ID                             
C_1   1101    NaN  173.0    NaN  63.0
      1102  192.0    NaN   73.0   NaN
C_2   1201    NaN  188.0    NaN  68.0
      1202  176.0    NaN   94.0   NaN
C_3   1301    NaN  161.0    NaN  68.0
      1302  175.0    NaN   57.0   NaN
C_4   2401  192.0    NaN   62.0   NaN
      2402    NaN  166.0    NaN  82.0

In [30]:
df_stacked = df_s.stack()
df_stacked.groupby('Class').head(2)

Height  Weight
Class ID   Gender                
C_1   1101 M        173.0    63.0
      1102 F        192.0    73.0
C_2   1201 M        188.0    68.0
      1202 F        176.0    94.0
C_3   1301 M        161.0    68.0
      1302 F        175.0    57.0
C_4   2401 F        192.0    62.0
      2402 M        166.0    82.0

- stack函数可以看做将横向的索引放到纵向，因此功能类似与melt，参数level可指定变化的列索引是哪一层（或哪几层，需要列表）

In [32]:
df_stacked = df_s.stack(0)
df_stacked.groupby('Class').head(2)

Gender                 F      M
Class ID                       
C_1   1101 Height    NaN  173.0
           Weight    NaN   63.0
C_2   1201 Height    NaN  188.0
           Weight    NaN   68.0
C_3   1301 Height    NaN  161.0
           Weight    NaN   68.0
C_4   2401 Height  192.0    NaN
           Weight   62.0    NaN

#### 2) unstack：stack的逆函数，功能上类似于pivot_table

In [33]:
df_stacked.head()

Gender                 F      M
Class ID                       
C_1   1101 Height    NaN  173.0
           Weight    NaN   63.0
      1102 Height  192.0    NaN
           Weight   73.0    NaN
      1103 Height    NaN  186.0

In [34]:
result = df_stacked.unstack().swaplevel(1,0,axis=1).sort_index(axis=1)
result.equals(df_s)
#同样在unstack中可以指定level参数

True

### 三、哑变量与因子化
#### 1. Dummy Variable（哑变量）
- 这里主要介绍get_dummies函数，其功能主要是进行one-hot编码：

In [35]:
df_d = df[['Class','Gender','Weight']]
df_d.head()

,Class,Gender,Weight
0,C_1,M,63
1,C_1,F,73
2,C_1,M,82
3,C_1,F,81
4,C_1,F,64


- 现在希望将上面的表格前两列转化为哑变量，并加入第三列Weight数值：

In [36]:
pd.get_dummies(df_d[['Class','Gender']]).join(df_d['Weight']).head()
#可选prefix参数添加前缀，prefix_sep添加分隔符

,Class_C_1,Class_C_2,Class_C_3,Class_C_4,Gender_F,Gender_M,Weight
0,1,0,0,0,0,1,63
1,1,0,0,0,1,0,73
2,1,0,0,0,0,1,82
3,1,0,0,0,1,0,81
4,1,0,0,0,1,0,64


#### 2. factorize方法
- 该方法主要用于自然数编码，并且缺失值会被记做-1，其中sort参数表示是否排序后赋值

In [37]:
codes, uniques = pd.factorize(['b', None, 'a', 'c', 'b'], sort=True)
display(codes)
display(uniques)

array([ 1, -1,  0,  2,  1], dtype=int64)

array(['a', 'b', 'c'], dtype=object)

### 四、问题与练习
### 1. 问题

【问题一】 上面提到了许多变形函数，如melt/crosstab/pivot/pivot_table/stack/unstack函数，请总结它们各自的使用特点。

#### 1.melt函数：
- pandas.melt(frame, id_vars=None, value_vars=None, var_name=None, value_name='value', col_level=None)
#### 参数解释：
- frame:要处理的数据集。
- id_vars:不需要被转换的列名。
- value_vars:需要转换的列名，如果剩下的列全部都要转换，就不用写了。
- var_name和value_name是自定义设置对应的列名。
- col_level :如果列是MultiIndex，则使用此级别。

In [2]:
import numpy as np
import pandas as pd

cheese = pd.DataFrame({'first': ['John', 'Mary'],
                       'last': ['Doe', 'Bo'],
                       'height': [5.5, 6.0],
                       'weight': [130, 150]})

print(cheese)

  first last  height  weight
0  John  Doe     5.5     130
1  Mary   Bo     6.0     150


In [3]:
# eg1.idenfier变量为first和last（其他的列默认为measured变量）
print(cheese.melt(id_vars=['first', 'last']))

  first last variable  value
0  John  Doe   height    5.5
1  Mary   Bo   height    6.0
2  John  Doe   weight  130.0
3  Mary   Bo   weight  150.0


In [4]:
# eg2.自定义variable和value列的名字
print(cheese.melt(id_vars=['first', 'last'], var_name='myVariable', value_name='myValue'))

  first last myVariable  myValue
0  John  Doe     height      5.5
1  Mary   Bo     height      6.0
2  John  Doe     weight    130.0
3  Mary   Bo     weight    150.0


#### 2.stack和unstack函数：
- stack和unstack是用来操作MultiIndex的。
- 其实多重索引相当于用二维的形式来表示高维数据。
- 作为二维数据的行和列都可以来做多重，stack和unstack就是在多重行索引和多重列索引（列名）之间转化的。

In [5]:
import pandas as pd
import numpy as np

tuples = list(zip(*[['bar', 'bar', 'baz', 'baz',
                     'foo', 'foo', 'qux', 'qux'],
                    ['one', 'two', 'one', 'two',
                     'one', 'two', 'one', 'two']]))

index = pd.MultiIndex.from_tuples(tuples, names=['first', 'second'])
df = pd.DataFrame(np.random.randn(8,2), index=index, columns=['A', 'B'])
print(df)

                     A         B
first second                    
bar   one    -0.444751  0.218916
      two    -2.102599  0.555432
baz   one    -1.051936  0.870550
      two     0.988654  0.934795
foo   one     0.231763 -0.169860
      two    -0.445636  1.455465
qux   one    -0.334762  0.238287
      two    -0.438654 -0.260893


In [6]:
# eg1.stack方法可以来"压缩"一下列索引，这可能产生一个Series（如果本身列所以那就是一重的），或者一个DataFrame（列是多重索引）
# 这里的结果就是产生了一个Series，行索引是三重的。
df1 = df.stack()
print(df1)
print(df1.index)

first  second   
bar    one     A   -0.444751
               B    0.218916
       two     A   -2.102599
               B    0.555432
baz    one     A   -1.051936
               B    0.870550
       two     A    0.988654
               B    0.934795
foo    one     A    0.231763
               B   -0.169860
       two     A   -0.445636
               B    1.455465
qux    one     A   -0.334762
               B    0.238287
       two     A   -0.438654
               B   -0.260893
dtype: float64
MultiIndex([('bar', 'one', 'A'),
            ('bar', 'one', 'B'),
            ('bar', 'two', 'A'),
            ('bar', 'two', 'B'),
            ('baz', 'one', 'A'),
            ('baz', 'one', 'B'),
            ('baz', 'two', 'A'),
            ('baz', 'two', 'B'),
            ('foo', 'one', 'A'),
            ('foo', 'one', 'B'),
            ('foo', 'two', 'A'),
            ('foo', 'two', 'B'),
            ('qux', 'one', 'A'),
            ('qux', 'one', 'B'),
            ('qux', 'two', 'A'),
         

In [7]:
#eg2.相反的操作是unstack()，即减少一重行索引，增加一重列索引
df2 = df1.unstack()
print(df2)

                     A         B
first second                    
bar   one    -0.444751  0.218916
      two    -2.102599  0.555432
baz   one    -1.051936  0.870550
      two     0.988654  0.934795
foo   one     0.231763 -0.169860
      two    -0.445636  1.455465
qux   one    -0.334762  0.238287
      two    -0.438654 -0.260893


#### 3.pivot 函数
- pivot做的就是从源数据构造透视表的过程，透视表是在数据统计中一个数据汇总的方式

#### 4.pivot_table 函数
- pivot_table根据文档上的解释是可以create一些spreadsheet-style类型的数据表。其实对于简单DataFrame来说和pivot类似。但是在那基础上增加了更多的一些功能。
#### 主要参数如下:
- data：DataFrame
- index：index列，可以多个。pivot不可以。
- columns：column列
- values：value列
- aggFuc：聚合函数，可以是多个。相同的index和columns确定的值如果有多个，则根据这个函数计算除一个值作为这个index和columns的值。默认情况是np.mean！，pivot在这种情况会提示有重复值，不能处理这种情况。

#### 5.crosstab 函数
#### 典型的用途如分组统计,函数形式如下
- (index,columns,values=None,rownames=None,colnames=None,aggfunc=None,margins=False,dropna=true) 
#### 参数：关键的参数有三个，分别是index,columns,values
- index相当于是行，
- columns相当于列，
- values相当于要做统计的对象

【问题二】 变形函数和多级索引是什么关系？哪些变形函数会使得索引维数变化？具体如何变化？

#### 1.变形函数相当于对多级索引进行重排
首先需要理解索引维度如何界定，所有不同维度的形式如下：
- 0维，又称0维张量，数字，标量：1
- 1维，又称1维张量，数组，vector：[1, 2, 3]
- 2维，又称2维张量，矩阵，二维数组：[[1,2], [3,4]]
- 3维，又称3维张量，立方（cube），三维数组：[ [[1,2], [3,4]], [[5,6], [7,8]] ]
- n维：,,,,,,

再多的维只不过是是把上一个维度当作自己的元素，1维的元素是标量，2维的元素是数组，3维的元素是矩阵。从0维到3维，边看边念咒语“维度是用来索引一个多维数组中某个具体数所需要最少的坐标。”

#### 2.crosstab,stack和unstack,melt，pivot_table函数都可以使得索引维数变化

#### 3.pandas生成透视表后，index会变成multiindex，即多层索引

In [ ]:
【问题三】 请举出一个除了上文提过的关于哑变量方法的例子。

#### 哑变量我没理解透，还需要再研究下
- 哑变量（Dummy Variable），又称为虚拟变量、虚设变量或名义变量，从名称上看就知道，它是人为虚设的变量，通常取值为0或1，来反映某个变量的不同属性。对于有n个分类属性的自变量，通常需要选取1个分类作为参照，因此可以产生n-1个哑变量。将哑变量引入回归模型，虽然使模型变得较为复杂，但可以更直观地反映出该自变量的不同属性对于因变量的影响，提高了模型的精度和准确度。

#### 对于无序多分类变量，引入模型时需要转化为哑变量：
- 例如血型，一般分为A、B、O、AB四个类型，为无序多分类变量，通常情况下在录入数据的时候，为了使数据量化，我们常会将其赋值为1、2、3、4。从数字的角度来看，赋值为1、2、3、4后，它们是具有从小到大一定的顺序关系的，而实际上，四种血型之间并没有这种大小关系存在，它们之间应该是相互平等独立的关系。如果按照1、2、3、4赋值并带入到回归模型中是不合理的，此时我们就需要将其转化为哑变量。


In [ ]:
【问题四】 使用完stack后立即使用unstack一定能保证变化结果与原始表完全一致吗？

#### 一定能保证变化结果是与原始表可以完全一致的
下面从两个函数的定义来看，
- 1.stack()是将原来的列索引转成了最内层的行索引，这里是多层次索引，其中合并的列索引对应第三层，即最内层索引。
- 2.unstack()是stack()的逆操作，这里把最内层的行索引还原成了列索引。但是unstack()中有一个参数可以指定旋转第几层索引，比如unstack(0)就是把第一层行索引转成列索引，但默认的是把最内层索引转层列索引。

#### 所以unstack()中参数指定旋转索引层数与stack()合并的列索引对应层数一致，就可以保证完全一致。

【问题五】 透视表中涉及了三个函数，请分别使用它们完成相同的目标（任务自定）并比较哪个速度最快。

In [27]:
%timeit df.pivot(index='ID',columns='Gender',values='Height')
%timeit pd.pivot_table(df,index='ID',columns='Gender',values='Height')
%timeit pd.crosstab(index=df['ID'],columns=df['Gender'],values='Height',aggfunc='sum' )

1.3 ms ± 2.6 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
5.36 ms ± 39.9 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
10.3 ms ± 11.3 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


#### 根据比较结果来看，pivot 最快，pivot_table 次之，crosstab 最慢

In [ ]:
【问题六】 既然melt起到了stack的功能，为什么再设计stack函数？

- stack是用来操作MultiIndex的，stack和unstack主要是在多重行索引和多重列索引之间转化。

### 2. 练习

In [ ]:
【练习一】 继续使用上一章的药物数据集：

In [28]:
pd.read_csv('joyful-pandas-master/data/Drugs.csv').head()

,YYYY,State,COUNTY,SubstanceName,DrugReports
0,2010,VA,ACCOMACK,Propoxyphene,1
1,2010,OH,ADAMS,Morphine,9
2,2010,PA,ADAMS,Methadone,2
3,2010,VA,ALEXANDRIA CITY,Heroin,5
4,2010,PA,ALLEGHENY,Hydromorphone,5


#### a.现在请你将数据表转化成如下形态，每行需要显示每种药物在每个地区的10年至17年的变化情况，且前三列需要排序：

In [36]:
#### pd.pivot_table(df,index=['State','COUNTY','SubstanceName'],columns='YYYY',values='DrugReports')
#### 练习对小白来说，需要时间，实在来不及做了，等五一做综合练习把，哈哈，先记录个答案，以后自己复习方便。
result = pd.pivot_table(df,index=['State','COUNTY','SubstanceName']
                 ,columns='YYYY'
                 ,values='DrugReports',fill_value='-').reset_index().rename_axis(columns={'YYYY':''})
result.head()

#### b.现在请将(a)中的结果恢复到原数据表，并通过equal函数检验初始表与新的结果是否一致（返回True）

In [ ]:
#### 练习对小白来说，需要时间，实在来不及做了，等五一做综合练习把，哈哈，先记录个答案，以后自己复习方便。
result_melted = result.melt(id_vars=result.columns[:3],value_vars=result.columns[-8:]
                ,var_name='YYYY',value_name='DrugReports').query('DrugReports != "-"')
result2 = result_melted.sort_values(by=['State','COUNTY','YYYY'
                                    ,'SubstanceName']).reset_index().drop(columns='index')
#下面其实无关紧要，只是交换两个列再改一下类型（因为‘-’所以type变成object了）
cols = list(result2.columns)
a, b = cols.index('SubstanceName'), cols.index('YYYY')
cols[b], cols[a] = cols[a], cols[b]
result2 = result2[cols].astype({'DrugReports':'int','YYYY':'int'})
result2.head()

In [ ]:
【练习二】 现有一份关于某地区地震情况的数据集，请解决如下问题：

In [31]:
pd.read_csv('joyful-pandas-master/data/Earthquake.csv').head()

,日期,时间,维度,经度,方向,距离,深度,烈度
0,2003.05.20,12:17:44 AM,39.04,40.38,west,0.1,10.0,0.0
1,2007.08.01,12:03:08 AM,40.79,30.09,west,0.1,5.2,4.0
2,1978.05.07,12:41:37 AM,38.58,27.61,south_west,0.1,0.0,0.0
3,1997.03.22,12:31:45 AM,39.47,36.44,south_west,0.1,10.0,0.0
4,2000.04.02,12:57:38 AM,40.80,30.24,south_west,0.1,7.0,0.0


#### a.现在请你将数据表转化成如下形态，将方向列展开，并将距离、深度和烈度三个属性压缩：

In [ ]:
#### 练习对小白来说，需要时间，实在来不及做了，等五一做综合练习把，哈哈，先记录个答案，以后自己复习方便。
result = pd.pivot_table(df,index=['日期','时间','维度','经度']
            ,columns='方向'
            ,values=['烈度','深度','距离'],fill_value='-').stack(level=0).rename_axis(index={None:'地震参数'})
result.head(6)

#### b.现在请将(a)中的结果恢复到原数据表，并通过equal函数检验初始表与新的结果是否一致（返回True）

In [ ]:
#### 练习对小白来说，需要时间，实在来不及做了，等五一做综合练习把，哈哈，先记录个答案，以后自己复习方便。
df_result.astype({'深度':'float64','烈度':'float64','距离':'float64'},copy=False).dtypes